# Pull Operations Description Notebook

## Table of Contents for this Notebook

- [Pull Operations Description Notebook](#Pull-Operations-Description-Notebook)
- [Table of Contents for this Notebook](#Table-of-Contents-for-this-Notebook)
- [Pull Operation Concepts](#Pull-Operation-Concepts)
	- [Overview](#Overview)
	- [Relation to original operations](#Relation-to-original-operations)
	- [Pull Operation Responses](#Pull-Operation-Responses)
	- [Pull Request Sequence Code Pattern](#Pull-Request-Sequence-Code-Pattern)
	- [Common Pull Operation Request Input Arguments](#Common-Pull-Operation-Request-Input-Arguments)
	- [OpenRequests](#OpenRequests)
	- [PullRequests](#PullRequests)
	- [CloseRequest](#CloseRequest)
- [Pull EnumerateInstances](#Pull-EnumerateInstances)
- [Pull Enumerate InstancePaths](#Pull-Enumerate-InstancePaths)
- [Pull Associator Instances](#Pull-Associator-Instances)
- [Pull Associator Paths](#Pull-Associator-Paths)
- [Pull Reference Instances](#Pull-Reference-Instances)
- [Pull Query Instances](#Pull-Query-Instances)




## Pull Operation Concepts

### Overview

The DMTF CIM/XML Pull operations allow the WBEM client to break the monolithic instance operations for requests that deliver multiple objects into multiple requests/responses executed as a sequence of requests to limit the size of individual responses.

NOTE: The pull operations were added to PyWBEM in version 0.9.0.

They were created to reduce scalability issues with extremely large responses from the enumerate requests for instances (EnumerateInstances, Associators, and References) that were causing resource problems in both clients and servers by allowing the client to break large responses up into multiple smaller responses and allowing some filter of the responses to be performed by the server.

A central concept of pulled enumeration operations is the `enumeration session`, which provides a context in which the operations perform their work and which determines the set of instances or instance paths to be returned. To process the operations of an enumeration session, some parameters of the open operation need to be maintained as long as the enumeration session is open. In addition, some state data about where the enumeration session is with regard to instances or instance paths already .returned must be maintained.

A successful `Open...` operation establishes the enumeration session and returns an enumeration context value representing it. This value is used as an input/output parameter in subsequent Pull operations on that enumeration session.

In general they replace the single request/response (ex. EnumerateInstances) that returns all instances in a single response  with a pattern that is based on two operations:

* `Open...` to open the request enumeration session to the WBEM Server and 
* `Pull...` to continue retrieving objects from the WBEM Server after a successful `Open...`. The client continues
to pull until an exception or end-of-sequence flag is received.

They use the same request parameters and the original operations and add several more parameters to control the flow of responses (response size, timeouts, etc.)

### Relation to original operations

The convention for the pull operation nameing was as follows 1) Prepend the original operation name with `Open` and
`Pull`, 2) Suffix the pull operations that return both instances and paths with `WithPath`, 3) Change the name suffix on operations that return path information from `Names` to `Paths` to reflect that these operations are returning 
complete instance paths with host and namespace included.  The `Exec` was dropped from the name for the
`OpenQueryInstances`.

The CIM/XML pull operations parallel to the previous CIM/XML operations as follows

*Original Operation*   | *Equivalent Pull Operations*
 :-                    |:-
EnumerateInstances     | OpenEnumerateInstances / PullInstancesWithPath 
EnumerateInstanceNames | OpenEnumerateInstancePaths / PullInstancesPaths
Associators            | OpenAssociatorInstances / PullInstancesWithPath 
AssociatorNames        | OpenAssociatorPaths / PullInstancesPaths
References             | OpenReferenceInstances / PullInstancesWithPath 
ReferenceNames         | OpenReferencesPaths / PullInstancesPaths
ExecQuery              | OpenQueryInstances / PullInstances

The pull operations are defined only for instances.  There are NO pull operations for CIMClasses, the
CIMQualifierDeclarations or for the InvokeMethod

### Pull Operation Responses

Each pull operation request returns a Python named tuple result that consists of the following named components:    

* *`eos`* - A boolean flag that indicates end-of-sequence. As along as this returned flag is True, the server has more objects to return.

* `context` - An opaque identifier that must be returned to the server with subsequent pull requests to continue the enumeration sequence. The context received with a response within an enumeration MUST be returned with the next request since the context may uniquely define not only the enumeration sequence but the segement to be returned in the next request

* `instances` or `*paths*` - A list of pywbem objects returned from the server.  The requests that demand instances return the `*instances*` entry in the tuple and those that request paths return paths in the `path` object in the tuple.

### Pull Request Sequence Code Pattern

Generally the pattern for requesting from a server using the Pull operations is as follows:

::

    # open the enumeration sequence
    result = open...(uri, ...)
        ... process the objects return in result.xx
    # while more objects exist in the server, loop to pull objects
    while not result.eos
        result = pull...(result.context, <MaxObjectCount>, ...)
            ... process the objects return in result.xx
            
The user opens the request with the open request and if that is successful, and does not return the end-of-sequence flag
the result (`eos`) executed the pull request to continue receiving objects within the enumeration sequence.  Each pull
request MUST include the enumeration context from the previous response (`context` in the result tuple).

The pull sequence may be terminated by executing a [`CloseEnumeration()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.CloseEnumerate)
to terminate the pull sequence.  However, this is optional and used only to close pull sequences before the `eos` has
been received.
    


### Common Pull Operation Request Input Arguments

The following are the request arguments that are common across all of the Pull requests.

#### OpenRequests

* `FilterQuery Language` and `FilterQuery` - These input parameters specify a filter query that acts as an additional restricting filter on the set of enumerated instances/paths returned. WBEM servers must support filter queries in pulled enumerations and must support the DMTF Filter Query Language(FQL, see  DMTF DSP0212) as a query language. If a WBEM server accepts a request with the `FilterQuery` parameter defined it MUST filter the response.  NOTE: The query and query language defined for the `OpenQueryInstances` is NOT FQL but the same query languages defined for the execQuery request. 

* `OperationTimeout` - Determines the minimum time the WBEM server shall maintain the opened enumeration session after the last Open or Pull operation (unless the enumeration session is closed during the last operation). If the operation timeout is exceeded, the WBEM server may close the enumeration session at any time, releasing any resources allocated to the enumeration session. An OperationTimeout of 0 means that there is no operation timeout. That is, the enumeration session is never closed based on time. If OperationTimeout is NULL, the WBEM server shall choose an operation timeout.

* `ContinueOnError` - This input parameter, if true, requests a continuation on error, which is the ability to resume an enumeration session successfully after a Pull operation returns an error. If a WBEM server does not support continuation on error and ContinueOnError is true, it shall return a failure with the status code CIM_ERR_CONTINUATION_ON_ERROR_NOT_SUPPORTED.

* `MaxObjectCount` - Defines the maximum number of instances or instance paths that the open operation can return. Any uint32 number is valid, including 0. The WBEM server may deliver any number of instances or instance paths up to MaxObjectCount but shall not deliver more than MaxObjectCount elements. The default for this is zero so that the WBEM server does not deliver objects in the response unless a MaxObjectCount is specifically defined.

#### PullRequests

* `Context` - This is the EnumerationContext defined in the specification. It is an opaque string returned from the previous open or pull for this enumeration sequence as part of the result tuple (result.context).

* `MaxObjectCount` - This  required input parameter defines the maximum number of instances or instance paths that may be returned by this Pull operation. Any uint32 number is valid, including 0. The WBEM server may deliver any number of instances or instance paths up to MaxObjectCount but shall not deliver more than MaxObjectCount. The WBEM client may use a MaxObjectCount value of 0 to restart the operation timeout for the enumeration session when it does not need to not retrieve any instances or instance paths.

#### CloseRequest
* `Context` - This is the EnumerationContext defined in the specification. It is an opaque string returned from the previous open or pull for this enumeration sequence as part of the result tuple (result.context).

The pull operations differ from the original operations in the several ways:

1. They allow filtering the response in the WBEM Server which can represent a significant resource saving if only selected instances from a large response are really required.
2. They limit the amount of memory used by the server since the server need not process the complete request before returning information to the client
2. They limit the memory used by the client since it can define the maximum size of any request.
3. They allow the client to terminate a request early with the `CloseEnumeration`
4. They allow the server and client to receive partial responses in that the client receives potentially an error response on each segment of the response, not the overall response.
5. They provide a more consistent inclusion of the path component in the responses.

<a href="#" onclick="history.back()">&lt;--- Back</a>

## Pull EnumerateInstances

The PullEnumerateInstances request operation sequence differs from the [`EnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstances) request in that its consists of multiple of operations, an [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) operation and corresponding [`PullInstancesWithPath()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstancesWithPath) operations. The [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) opens an enumeration sequence and optionally acquires instances from the server determined by the `MaxObjectCount` parameter.  It returns a named tuple that defines the sequence, marks, the end of the sequence, and includes instances retrieved by the [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) operation.

Subsequent [`PullEnumerateInstancesWithPath()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstancesWithPath) operations retrieve more instances as defined by the `MaxObjectCount` parameter of the call until the end-of-sequence flag (result.eos) is received.

The [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) method returns a Python named tuple with the components as defined above:

* `eos`
* `context`
* `instances` - A list of [`pywbem.CIMInstance`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstance) objects for each CIM instance of a particular CIM class (and its subclasses). The CIM instance path is part of the returned instance objects and can be accessed via their `path` attribute returned for this request operation. The instances may be a partial or complete response depending on the result.eos flag.

The [`PullEnumerateInstancesWithPath()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstancesWithPath) method returns a named tuple containing the same named elements as the [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) above (eos, context, instances). Note that while the
MaxObjectCount parameter is an option with the open request (the default is zero) it is a required parameter with the
pull request.

The following code enumerates the instances of the specified CIM class with an open and subsequent pull requests and prints their instance paths in WBEM URI format (see  [`pywbem.CIMInstanceName.__str__()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstanceName.__str__)), and the instance itself in MOF format (see [`pywbem.CIMInstance.tomof()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.CIMInstance.tomof)) when all have been retrieved.

The example sets the maximum number of instances that will be retrieved for each request including the open (MaxObjectCount) at 100. Each response may return 100 or fewer instances. 

In this code, the received results are accumulated into the `insts` list.  They could also be processed directly after
each response is received. The only caution is that if they are processed immediatly, the client must understand that
the response sequence could end early and incomplete if an exception is received and the client must account for this incomplete response in its processing.

In [ ]:
from __future__ import print_function
import pywbem

username = 'user'
password = 'password'
classname = 'PyWBEM_Person'
namespace = 'root/cimv2'
server = 'http://localhost'
max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    result = conn.OpenEnumerateInstances(classname, MaxObjectCount=max_obj_cnt)
    insts = result.instances
    while not result.eos:
        result = conn.PullInstancesWithPath(result.context, max_obj_cnt)
        insts.extend(result.instances)                                    
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s instances' % (len(insts)))
    for inst in insts:
        print('path=%s' % inst.path)
        print(inst.tomof())

In this example, the connection has a default namespace set. This allows us to omit the namespace from the subsequent [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstances) method.

This example also shows exception handling with PyWBEM: PyWBEM wraps any exceptions that are considered runtime errors, and raises them as subclasses of [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error). Any other exceptions are considered programming errors. Therefore, the code above only needs to catch [`pywbem.Error`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.Error).

Note that the creation of the [`pywbem.WBEMConnection`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection) object in the code above does not need to be protected by exception handling; its initialization code does not raise any PyWBEM runtime errors.

## Pull Enumerate InstancePaths

The Pull EnumerateInstancePaths operation parallels the Pull EnumerateInstances sequence except that it returns instance paths instead of instances.

The PullEnumerateInstances request operation sequence differs from the [`EnumerateInstanceNames()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstanceNames) request in that its consists of multiple of operations, an [`OpenEnumerateInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstancePaths) operation and corresponding [`PullEnumerateInstancesPaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstancePaths) operations. The [`OpenEnumerateInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.EnumerateInstances) opens an enumeration sequence and optionally acquires instance paths from the server determined by the `MaxObjectCount` parameter.  It returns a named tuple that defines the sequence, marks, the end of the sequence, and includes instance paths retrieved by the [`OpenEnumerateInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenEnumerateInstancePaths) operation.


In [ ]:
from __future__ import print_function
import pywbem

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    result = conn.OpenEnumerateInstancePaths(classname, MaxObjectCount=max_obj_cnt)
    paths = result.paths
    while not result.eos:
        result = conn.PullInstancesPaths(result.context, max_obj_cnt)
        insts.extend(result.paths)                                    
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s instances' % (len(paths)))
    for path in paths:
        print('path=%s' % path)
  

## Pull Associator Instances

TODO Document

In [ ]:

max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        result = conn.OpenAssociatorInstances(source_paths[0], MaxObjectCount=max_obj_cnt)
        insts = result.instances
        while not result.eos:
            result = conn.PullInstancesWithPath(result.context, max_obj_cnt)
            insts.extend(result.instances)
    else:
        print('%s class has no instances and therefore no associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s instance(s)' % (len(insts)))
    for inst in insts:
        print('path=%s' % inst.path)
        print(inst.tomof())

## Pull Associator Paths

In [ ]:
TODO add documentation

In [ ]:
max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        result = conn.OpenAssociatorInstancePaths(source_paths[0], MaxObjectCount=max_obj_cnt)
        paths = result.paths
        while not result.eos:
            result = conn.PullInstancePaths(result.context, max_obj_cnt)
            insts.extend(result.paths)
    else:
        print('%s class has no paths and therefore no associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s paths' % (len(paths)))
    for path in paths:
        print('path=%s' % path)

## Pull Reference Instances

In [ ]:
TODO add Doc

In [ ]:
max_obj_cnt = 100

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        result = conn.OpenReferenceInstances(source_paths[0], MaxObjectCount=max_obj_cnt)
        insts = result.instances
        while not result.eos:
            result = conn.PullInstancesWithPath(result.context, max_obj_cnt)
            insts.extend(result.instances)
    else:
        print('%s class has no instances and therefore no associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s instances\n' % (len(insts)))
    for inst in insts:
        print('path=%s\n' % inst.path)
        print(inst.tomof())

In [ ]:
## Pull Reference InstancePaths

In [ ]:
The pull reference instance paths enumeration sequence is opened with an 
[`OpenReferenceInstancePaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenreferenceInstancePaths)
operation and corresponding [`PullInstancesEithPaths()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstancesWithPath)

The example below first gets instance names from a class that is part of an association and then uses one of those
instances to execute the OpenReferenceInstances operation. Normally the code should always include the full pattern of
the Open followed by the Pull loop because the client cannot be certain how many objects will be returned in the
response to the open request (the specification only requires that no more than MaxObjectCount be returned). In fact,
some WBEM server implementations recommend that the client delay slightly between a response and subsequent pull to insure
the server has more objects prepared to send.
                                                         
The extra print statements in the example below record the the responses received.

In [ ]:
max_open_cnt = 0    # no instance paths returned on open
max_pull_cnt = 1    # zero or one instance path returned for each pull

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    # Find an instance path for the source end of an association.
    source_paths = conn.EnumerateInstanceNames(classname)
    if source_paths:
        result = conn.OpenReferenceInstancePaths(source_paths[0], MaxObjectCount=max_open_cnt)
        paths = result.paths
        print('Open eos=%s, context=%s, path count %s' % (result.eos, result.context, len(result.paths)))
        while not result.eos:
            result = conn.PullInstancePaths(result.context, max_pull_cnt)
            print('Pull eos=%s, context=%s, path count %s' % (result.eos, result.context, len(result.paths)))
            paths.extend(result.paths)
    else:
        print('%s class has no paths and therefore no associations', classname)
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)
else:
    print('Retrieved %s paths' % (len(paths)))
    for path in paths:
        print('path=%s' % path)

## Pull Query Instances

In [ ]:
The pull query instances enumeration sequence is a parallel to the original ExecQuery operation.  It requests that the
server execute a query defined by the query parameter and the query language parameters.   These are NOT the same query
and query language as the other pull operations. The query language is normally WQL or the DMTF CQL.

The [`OpenQueryInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.OpenQueryInstances)
defines the query language and query required parameters.
                             
This enumeration sequence uses the [`PullInstances()`](https://pywbem.readthedocs.io/en/latest/client.html#pywbem.WBEMConnection.PullInstances)
operation because the query function returns instances without paths.
                                                       
The following example execute a simple

In [ ]:
from __future__ import print_function
import pywbem

username = 'user'
password = 'password'
classname = 'Pywbem_Person'
namespace = 'root/cimv2'
server = 'http://localhost'
max_object_cnt = 10

query_language = "DMTF:CQL"
query = 'Select  * from Pywbem_Person'

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)

try:
    result = conn.OpenQueryInstances('DMTF:CQL',
                                     query,
                                     MaxObjectCount=max_object_cnt)
    insts = result.instances
    while not result.eos:
        result = conn.PullInstances(result.context, max_object_cnt)
        insts.extend(result.instances)
                                  
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

In [ ]:
## CloseEnumeration() Request

The following is an example of using the CloseEnumeration request to terminate an enumeration sequence before it has completed. It requests zero responses from the open and terminates before the first pull. In this example, since the MaxObjectCount for the open is zero, no instances should be processed before the close is issued.

In [ ]:
from __future__ import print_function
import pywbem

username = 'user'
password = 'password'
classname = 'CIM_ComputerSystem'
namespace = 'root/cimv2'
server = 'http://localhost'

conn = pywbem.WBEMConnection(server, (username, password),
                             default_namespace=namespace,
                             no_verification=True)
try:
    result = conn.OpenEnumerateInstances(classname, MaxObjectCount=0)
    if result.eos:      
        result = conn.CloseEnumeration(result.context)
    print('instance count = %s' % len(result.instances))
                                  
except pywbem.Error as exc:
    print('Operation failed: %s' % exc)

<a href="#" onclick="history.back()">&lt;--- Back</a>